In [1]:
def upper_right(N,q,y):
    from scipy.linalg import circulant
    def alpha(N,q,y):
        import math
        L = []
        k = int((N-1)/2)
        #print(k)
        for j in range(k):
            L.append(j-k)
        for j in range(k,N-2+1):
            L.append(j+1-k)
        s = math.floor(N*q**(1/y)) + 1
        L.append(s)
        return L
    L = alpha(N,q,y)
    return circulant(L).T

def upper_left(N):
    from scipy.sparse import identity
    return identity(N).toarray().astype(int)

def bottom_left(N):
    import numpy as np
    return np.zeros((N,N)).astype(int)

def bottom_right(N,q):
    from scipy.sparse import identity
    return q*identity(N).toarray().astype(int)

def ntru_matrix(N,q,y):
    import numpy as np
    return np.block([
    [upper_left(N), upper_right(N,q,y)],

    [bottom_left(N), bottom_right(N,q)]])

def mat2fp(A):
    from fpylll import IntegerMatrix
    L = IntegerMatrix(A.shape[0],A.shape[1])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            L[i,j] = int(A[i,j])
    return L

# converts from type fpylll.integetMatrix to numpy.array
def fp2mat(A):
    L = np.zeros(shape=(A.nrows, A.ncols))
    for i in range(A.nrows):
        for j in range(A.ncols):
            L[i, j] = int(A[i, j])
            L = np.array(L).astype(int)
    return L


def lllreduce(A):
    from fpylll.fplll.lll import LLLReduction,LLL
    from fpylll import FPLLL
    _ = FPLLL.set_precision(53)
    A_fp = mat2fp(A)
    print(A_fp.nrows,"X",A_fp.ncols)
    LLL.reduction(A_fp, delta =0.99 )
    print("Is reduced?",LLL.is_reduced(A_fp))
    
    return A_fp

In [62]:
from fpylll.fplll.lll import LLLReduction,LLL
from fpylll import SVP
from numpy import linalg as LA
import numpy as np
for N,q,y in [[21,2**8,20],[23,2**8,27],[25,2**9,13], [27,2**9,15]]:
    print(N,q,y)
    print("1st (by heuristic):",q**(1/y)*(N+1)>0.35*np.sqrt(q))
    import math
    s=math.floor(N*q**(1/y)) + 1
    print("2nd:",(N**2-1)*N/12 + s**2<1/N * (q-q**(1/y))**2 / q**(2/y))
    A = ntru_matrix(N,q,y)
    A_fp = lllreduce(A)
    LLL.is_reduced(A_fp)
    _=SVP.shortest_vector(A_fp)
    left_part = N*q**(1/y)
    # the next line calls SVP function
    shortest = A_fp[0].norm()
    print("shortest:",shortest,"Nq^(1/y):",left_part)
    print("1st:",left_part<shortest)
    print("\n" )
    

21 256 20
1st (by heuristic): True
2nd: True
42 X 42
Is reduced? True
shortest: 29.017236257093817 Nq^(1/y): 27.70966612623078
1st: True


23 256 27
1st (by heuristic): True
2nd: True
46 X 46
Is reduced? True
shortest: 30.0 Nq^(1/y): 28.243720743439447
1st: True


25 512 13
1st (by heuristic): True
2nd: True
50 X 50
Is reduced? True
shortest: 43.520110293977886 Nq^(1/y): 40.39665360072864
1st: True


27 512 15
1st (by heuristic): True
2nd: True
54 X 54
Is reduced? True
shortest: 42.61455150532503 Nq^(1/y): 40.92434729578075
1st: True


